# Logistic Regression for Sentiment Analysis on Tweets

In [1]:
import re
import string
import nltk
import numpy as np
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [2]:
# Tập dữ liệu ví dụ

train_x = [
           'just plain boring',
           'entirely predictable and lacks energy',
           'no surprises and very few laughs',
           'very powerful',
           'the most fun film of the summer'
]
train_y = [0, 0, 0, 1, 1]

**Download Dataset**

In [3]:
# Tải về tập dữ liệu tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# Chia thành 2 tập train và test
# train: 4000 samples, test: 1000 samples
train_pos = all_positive_tweets[:4000]
test_pos = all_positive_tweets[4000:]

train_neg = all_negative_tweets[:4000]
test_neg = all_negative_tweets[4000:]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# Tạo nhãn negative: 0, positive: 1
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

**Tiền xử lý dữ liệu cho tập Tweets**

In [4]:
def basic_preprocess(text):
    '''
    Args:
        text: câu đầu vào
    Output:
        text_clean: danh sách các từ (token) sau khi chuyển sang chữ thường và
            được phân tách bởi khoảng trắng
    '''
    # xóa bỏ stock market tickers like $GE
    text = re.sub(r'\$\w*', '', text)

    # xóa bỏ old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)

    # xóa bỏ hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)

    # xóa bỏ hashtags
    text = re.sub(r'#', '', text)

    # tokenize
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    text_clean = []
    for word in text_tokens:
        if word not in string.punctuation:  # remove punctuation
            text_clean.append(word)

    return text_clean

In [5]:
# Kiểm tra kết quả
example_sentence = "RT @Twitter @chapagain Hello There! Have a great day. #good #morning http://chapagain.com.np"
basic_preprocess(example_sentence)

['hello', 'there', 'have', 'a', 'great', 'day', 'good', 'morning']

**Xây dựng bộ từ điển**

In [6]:
def count_freq_words(corpus, labels):
    """ Xây dựng bộ từ điển tần suất xuất hiện của các từ
    Args:
        corpus: tập danh sách các câu
        labels: tập nhãn tương ứng với các câu trong corpus (0 hoặc 1)
    Output:
        model: bộ từ điển ánh xạ mỗi từ và tần suất xuất hiện của từ đó trong corpus
            key: (word, label)
            value: frequency
            VD: {('boring', 0): 2} => từ boring xuất hiện 2 lần trong các sample thuộc class 0
    """
    model = {}
    for label, sentence in zip(labels, corpus):
        for word in basic_preprocess(sentence):
            # Định nghĩa key của model là tuple (word, label)
            pair = (word, label)
            # Nếu key đã tồn tại trong model thì tăng value lên 1
            if pair in model:
                model[pair] += 1
            # Nếu key chưa tồn tại trong model thì bổ sung key vào model với value =1
            else:
                model[pair] = 1
    return model

In [7]:
#Kiểm tra kết quả
freqs = count_freq_words(train_x, train_y)
freqs

{('followfriday', 1.0): 23,
 ('for', 1.0): 606,
 ('being', 1.0): 49,
 ('top', 1.0): 29,
 ('engaged', 1.0): 7,
 ('members', 1.0): 11,
 ('in', 1.0): 381,
 ('my', 1.0): 441,
 ('community', 1.0): 25,
 ('this', 1.0): 242,
 ('week', 1.0): 61,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('james', 1.0): 7,
 ('how', 1.0): 60,
 ('odd', 1.0): 1,
 (':/', 1.0): 5,
 ('please', 1.0): 77,
 ('call', 1.0): 21,
 ('our', 1.0): 111,
 ('contact', 1.0): 4,
 ('centre', 1.0): 1,
 ('on', 1.0): 242,
 ('02392441234', 1.0): 1,
 ('and', 1.0): 553,
 ('we', 1.0): 182,
 ('will', 1.0): 150,
 ('be', 1.0): 198,
 ('able', 1.0): 6,
 ('to', 1.0): 836,
 ('assist', 1.0): 1,
 ('you', 1.0): 1187,
 ('many', 1.0): 28,
 ('thanks', 1.0): 311,
 ('had', 1.0): 35,
 ('a', 1.0): 725,
 ('listen', 1.0): 8,
 ('last', 1.0): 36,
 ('night', 1.0): 50,
 ('as', 1.0): 82,
 ('bleed', 1.0): 2,
 ('is', 1.0): 354,
 ('an', 1.0): 99,
 ('amazing', 1.0): 39,
 ('track', 1.0): 5,
 ('when', 1.0): 69,
 ('are', 1.0): 152,
 ('scotland', 1.0): 2,
 ('congrats', 1.0)

In [8]:
# Hàm lấy ra tần suất xuất hiện là value trong `freq` dựa vào key (word, label)
def lookup(freqs, word, label):
    '''
    Args:
        freqs: một từ điển với tần số của mỗi cặp
        word: từ để tra cứu
        label: nhãn tương ứng với từ
    Output:
        count: số lần từ có nhãn tương ứng xuất hiện.
    '''
    count = 0

    pair = (word, label)
    if pair in freqs:
        count = freqs[pair]

    return count

In [9]:
#Kiểm tra kết quả
lookup(freqs, "just", 0)

197

**Logistic Regression**

In [10]:
def sigmoid(z): 
    '''
    Args:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    # calculate the sigmoid of z
    h = 1 / (1 + np.exp(-z))

    return h

In [11]:
def gradient_descent(x, y, theta, alpha, num_iters):
    '''
    Args:
        x: matrix of features, có chiều (m,n+1)
        y: label tương ứng (m,1)
        theta: vector trọng số (n+1,1)
        alpha: tốc độ học
        num_iters: số vòng lặp
    Output:
        J: final cost
        theta: vector trọng số
    '''
    # lấy m số lượng các sample trong matrix x
    m = len(x)
    
    for i in tqdm(range(num_iters)):
        
        # Tính z, phép dot product: x và theta
        z = np.dot(x, theta)
        
        # Tính h: sigmoid của z
        y_hat = sigmoid(z)
        
        # Tính cost function
        J = (-1 / m) * (np.dot(y.T, np.log(y_hat)) + np.dot((1 - y).T, np.log(1 - y_hat)))

        # Cập nhật trọn số theta
        theta = theta - (alpha / m) * (np.dot(x.T, (y_hat - y)))
        
    return J, theta

In [12]:
# Kiểm qua kết quả
np.random.seed(1)

# X input: 10 x 3, bias là 1
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)

# Y label: 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.5).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradient_descent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 100)
print(f"\nCost {tmp_J.item()}")
print(f"Weight {tmp_theta}")

100%|██████████| 100/100 [00:00<00:00, 13012.05it/s]


Cost 0.6860551249930995
Weight [[8.95483666e-08]
 [7.01794701e-05]
 [4.66109371e-05]]


**Trích xuất các feature**

In [13]:
def extract_features(text, freqs):
    '''
    Args: 
        text: tweet
        freqs: bộ từ điển tần suất xuất hiện của từ theo label (word, label)
    Output: 
        x: vector feature có chiều (1,3)
    '''
    # tiền xử lý
    word_l = basic_preprocess(text)
    
    # 3 thành phần: bias, feature 1 và feature 2
    x = np.zeros((1, 3)) 
    
    # bias
    x[0,0] = 1 

    for word in word_l:
        x[0,1] += lookup(freqs, word, 1)
        
        x[0,2] += lookup(freqs, word, 0)

    assert(x.shape == (1, 3))
    return x

In [14]:
# Kiểm tra
freqs = count_freq_words(train_x, train_y)
print(train_x[0])
extract_features(train_x[0], freqs)

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


array([[1.000e+00, 4.722e+03, 1.612e+03]])

In [15]:
# Kiểm tra
# VD: các từ không có trong bộ `freq`
x_test = "việt nam"
extract_features(x_test, freqs)

array([[1., 0., 0.]])

**Huấn luyện mô hình Logistic Regression**

In [16]:
# Tạo ma trận X có kích thước mxn với n=3 (số features)
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

Y = np.expand_dims(train_y, 1)

# Huấn luyện với số vòng lặp 1500, tốc độ học 1e-6
J, theta = gradient_descent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"Cost {J.item()}.")
print(f"Weight {theta}")

100%|██████████| 1500/1500 [00:05<00:00, 254.35it/s]

Cost 0.23335802523493712.
Weight [[ 5.85783372e-08]
 [ 5.70185881e-04]
 [-5.08632054e-04]]


**Dự đoán**

In [17]:
def predict_tweet(text, freqs, theta):
    '''
    Args: 
        text: tweet
        freqs: bộ từ điển tần suất xuất hiện của từ theo label (word, label)
        theta: (3,1) vector trọng số
    Output: 
        y_pred: xác suất dự đoán
    '''
  
    # extract features
    x = extract_features(text, freqs)
    
    # dự đoán
    y_pred = sigmoid(np.dot(x, theta))  
    
    return y_pred

In [18]:
tests = ["happy", "sad"]
for t in tests:
    pred = predict_tweet(t, freqs, theta)
    print(f'{t} -> {pred}')

happy -> [[0.51894153]]
sad -> [[0.48785679]]


**Đánh giá độ chính xác trên tập test**

In [19]:
acc = 0
for sentence, label in zip(test_x, test_y):

    # dự đoán từng câu trong tập test
    pred = predict_tweet(sentence, freqs, theta)

    if pred > 0.5:
        pred_l = 1
    else:
        pred_l = 0

    # so sánh nhãn dự đoán với nhãn thực tế
    if int(pred_l) == int(label):
        acc += 1

print('Accuracy: ', acc/len(test_x))

Accuracy:  0.967
